In [2]:
import cfbd
import numpy as np
import pandas as pd
import pickle
from sklearn import tree
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.ensemble import GradientBoostingClassifier as xgb

In [3]:
df = pd.read_csv('../cfb_imputed.csv')
test = pd.read_csv('../a_CFB_simple/df22.csv')
week4 = pd.read_csv('../a_CFB_simple/c.csv')
predictionsDB = pd.read_csv('../a_CFB_simple/d.csv')

In [4]:
df = df.drop('Unnamed: 0',axis=1)
df.head()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_explosiveness,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin
0,400763593,2015,1,False,UCF,American Athletic,14,1626,Florida International,Conference USA,...,1.397643,0.362319,0.233333,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,1
1,400603840,2015,1,True,South Carolina,SEC,17,1646,North Carolina,ACC,...,1.115637,0.461538,0.142857,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,-4
2,400763399,2015,1,False,Central Michigan,Mid-American,13,1417,Oklahoma State,Big 12,...,1.234668,0.426471,0.181818,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,11
3,400603839,2015,1,False,Vanderbilt,SEC,12,1365,Western Kentucky,Conference USA,...,1.001898,0.396226,0.380952,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,2
4,400756883,2015,1,False,Utah,Pac-12,24,1603,Michigan,Big Ten,...,1.163096,0.452055,0.241379,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,-7


In [5]:
y = []
for i in range(len(df)):
    if df.spread[i] < 0:
        if df.margin[i] > df.spread[i]:
            y.append(0)
        else:
            y.append(1)
    else:
        if df.margin[i] < df.spread[i]:
            y.append(1)
        else:
            y.append(0)
            
df["home_team_cover_spread"] = y
df.head()

,id,year,week,neutral_site,home_team,home_conference,home_points,home_elo,away_team,away_conference,...,d_successrate,d_stuffrate,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,margin,home_team_cover_spread
0,400763593,2015,1,False,UCF,American Athletic,14,1626,Florida International,Conference USA,...,0.362319,0.233333,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,1,0
1,400603840,2015,1,True,South Carolina,SEC,17,1646,North Carolina,ACC,...,0.461538,0.142857,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,-4,1
2,400763399,2015,1,False,Central Michigan,Mid-American,13,1417,Oklahoma State,Big 12,...,0.426471,0.181818,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,11,1
3,400603839,2015,1,False,Vanderbilt,SEC,12,1365,Western Kentucky,Conference USA,...,0.396226,0.380952,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,2,0
4,400756883,2015,1,False,Utah,Pac-12,24,1603,Michigan,Big Ten,...,0.452055,0.241379,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,-7,1


In [6]:
excluded = ['id','year','week','home_team','away_team','margin', 'home_points', 'away_points']
cat_features = ['home_conference','away_conference','neutral_site']
cont_features = [c for c in df.columns.to_list() if c not in cat_features and c not in excluded]

In [7]:
cats = df[cat_features]
cats = cats.apply(preprocessing.LabelEncoder().fit_transform)
cats

,home_conference,away_conference,neutral_site
0,1,4,0
1,9,0,1
2,6,2,0
3,9,4,0
4,8,3,0
...,...,...,...
4739,9,9,1
4740,0,0,1
4741,3,3,1
4742,8,8,0


In [8]:
df = df.drop(['neutral_site', 'home_conference', 'away_conference','id','week','home_team','away_team','margin', 'home_points', 'away_points'],axis=1)
df = df.join(cats)
df = df.drop('attendance', axis=1)
df.head()

,year,home_elo,away_elo,home_sp_plus,home_second_order_w,home_sos,home_special_teams_rating,home_d_r_explosivenenss,home_d_r_success,home_d_havoc_db,...,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,home_team_cover_spread,home_conference,away_conference,neutral_site
0,2015,1626,1321,-15.5,1.0,0.925,1.8,92.7,90.3,0.056,...,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,0,1,4,0
1,2015,1646,1477,3.4,4.1,0.820,0.8,105.5,87.6,0.039,...,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,1,9,0,1
2,2015,1417,1567,-3.1,7.2,0.935,-2.3,102.7,98.6,0.064,...,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,1,6,2,0
3,2015,1365,1521,-1.6,4.1,0.842,-0.1,98.3,112.2,0.085,...,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,0,9,4,0
4,2015,1603,1553,11.0,7.0,0.858,2.0,109.1,109.2,0.072,...,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,1,8,3,0


In [9]:
test_df = df.query("year == 2021")
train_df = df.query("year != 2021")
train_df.head()

,year,home_elo,away_elo,home_sp_plus,home_second_order_w,home_sos,home_special_teams_rating,home_d_r_explosivenenss,home_d_r_success,home_d_havoc_db,...,d_totalPPA,o_explosiveness,o_successrate,o_stuffrate,o_totalPPA,spread,home_team_cover_spread,home_conference,away_conference,neutral_site
0,2015,1626,1321,-15.5,1.0,0.925,1.8,92.7,90.3,0.056,...,15.894630,1.365478,0.406250,0.466667,4.270422,-17.0,0,1,4,0
1,2015,1646,1477,3.4,4.1,0.820,0.8,105.5,87.6,0.039,...,7.968428,1.130578,0.426667,0.170213,9.157397,-3.5,1,9,0,1
2,2015,1417,1567,-3.1,7.2,0.935,-2.3,102.7,98.6,0.064,...,18.608247,1.022896,0.416667,0.333333,8.286312,20.5,1,6,2,0
3,2015,1365,1521,-1.6,4.1,0.842,-0.1,98.3,112.2,0.085,...,4.754122,1.041876,0.397436,0.292683,-3.685131,-17.5,0,9,4,0
4,2015,1603,1553,11.0,7.0,0.858,2.0,109.1,109.2,0.072,...,-1.184796,1.085531,0.394366,0.250000,7.254696,-3.0,1,8,3,0


In [10]:
test_df.to_csv('cfb_15_21_test_wide.csv')
train_df.to_csv('cfb_15_21_train_wide.csv')

In [11]:
trainx = train_df.drop(['home_team_cover_spread', 'year'], axis=1)
testx = test_df.drop(['home_team_cover_spread', 'year'], axis=1)

In [12]:
trainy = train_df[["home_team_cover_spread"]]
testy = test_df[["home_team_cover_spread"]]

In [13]:
gb = xgb()

In [14]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(gb, trainx, trainy, scoring='accuracy', cv=cv, n_jobs=-1,error_score='raise')
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.793 (0.022)


In [15]:
model = xgb()

In [16]:
model.fit(trainx,trainy)

C:\Users\magamp\AppData\Roaming\Python\Python38\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()

In [17]:
preds = model.predict(testx)

In [18]:
confusion_matrix(testy,preds)

array([[293,  64],
       [ 62, 311]], dtype=int64)

In [19]:
# Week4 uses a subset of the data so we need a new model. This model should be trained on all games prior to week 4
week4 = week4.drop('Unnamed: 0',axis=1)
week4.columns


Index(['home_elo', 'away_elo', 'spread', 'h_drives', 'h_d_explosiveness',
       'h_d_successrate', 'h_d_stuffrate', 'h_d_totalPPA', 'h_o_explosiveness',
       'h_o_successrate', 'h_o_stuffrate', 'h_o_totalPPA', 'home_conference',
       'away_conference', 'neutral_site'],
      dtype='object')

In [20]:
week4.head()

,home_elo,away_elo,spread,h_drives,h_d_explosiveness,h_d_successrate,h_d_stuffrate,h_d_totalPPA,h_o_explosiveness,h_o_successrate,h_o_stuffrate,h_o_totalPPA,home_conference,away_conference,neutral_site
0,1446,1442,2.0,13.461538,1.331315,0.387627,0.209011,9.847823,1.249980,0.435223,0.188526,13.564157,0,2,0
1,1449,1665,2.5,12.600000,1.290190,0.413603,0.174861,12.324514,1.239361,0.427519,0.223501,13.328906,10,10,0
2,1542,1480,-10.0,13.750000,1.239019,0.448006,0.202265,15.622856,1.227273,0.392168,0.212778,8.339411,0,0,0
3,1684,1522,-24.0,11.303030,1.441883,0.377862,0.185552,9.415842,1.096152,0.476787,0.130066,17.938538,7,7,0
4,1251,1641,16.0,11.925926,1.238476,0.437122,0.197827,13.253949,1.422501,0.362134,0.238543,5.755227,4,7,0


In [21]:
df.columns

Index(['year', 'home_elo', 'away_elo', 'home_sp_plus', 'home_second_order_w',
       'home_sos', 'home_special_teams_rating', 'home_d_r_explosivenenss',
       'home_d_r_success', 'home_d_havoc_db', 'home_d_havoc_f7',
       'home_d_havoc_total', 'home_d_standardDowns', 'home_d_passing',
       'home_d_rushing', 'home_d_passingDowns', 'home_d_ranking',
       'home_o_r_explosiveness', 'home_o_r_success', 'home_o_standardDowns',
       'home_o_passing', 'home_o_rushing', 'home_o_runrate', 'home_o_pace',
       'home_o_passingDowns', 'home_o_ranking', 'away_sp_plus',
       'away_second_order_w', 'away_sos', 'away_special_teams_rating',
       'away_d_r_explosivenenss', 'away_d_r_success', 'away_d_havoc_db',
       'away_d_havoc_f7', 'away_d_havoc_total', 'away_d_standardDowns',
       'away_d_passing', 'away_d_rushing', 'away_d_passingDowns',
       'away_d_ranking', 'away_o_r_explosiveness', 'away_o_r_success',
       'away_o_standardDowns', 'away_o_passing', 'away_o_rushing',
       '

In [22]:
# Add the h_ suffix
dfx = df.copy()
dfx['h_drives'] = df['drives']
dfx['h_d_explosiveness']= df['d_explosiveness'] 
dfx['h_d_successrate']= df['d_successrate']
dfx['h_d_stuffrate']= df['d_stuffrate']
dfx['h_d_totalPPA']= df['d_totalPPA']
dfx['h_o_explosiveness']= df['o_explosiveness'] 
dfx['h_o_successrate']= df['o_successrate']
dfx['h_o_stuffrate']= df['o_stuffrate']
dfx['h_o_totalPPA']= df['o_totalPPA']
dfx['home_team_cover_spread'] = df['home_team_cover_spread']

In [23]:
w4trainx = dfx[week4.columns]
w4trainy = dfx['home_team_cover_spread']

In [35]:
w4_df =  dfx[week4.columns]
w4_df['home_team_cover_spread'] = dfx['home_team_cover_spread']
w4_df.to_csv('w4_data.csv')
# w4_df


C:\Users\magamp\AppData\Local\Temp\ipykernel_23544\2773481172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w4_df['home_team_cover_spread'] = dfx['home_team_cover_spread']


In [25]:
w4trainx.columns

Index(['home_elo', 'away_elo', 'spread', 'h_drives', 'h_d_explosiveness',
       'h_d_successrate', 'h_d_stuffrate', 'h_d_totalPPA', 'h_o_explosiveness',
       'h_o_successrate', 'h_o_stuffrate', 'h_o_totalPPA', 'home_conference',
       'away_conference', 'neutral_site'],
      dtype='object')

In [26]:
w4xgb = xgb()

In [27]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(w4xgb, w4trainx, w4trainy, scoring='accuracy', cv=cv, n_jobs=-1,error_score='raise')
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.801 (0.019)


In [28]:
w4xgb.fit(w4trainx,w4trainy)

GradientBoostingClassifier()

In [37]:
week4.head()

,home_elo,away_elo,spread,h_drives,h_d_explosiveness,h_d_successrate,h_d_stuffrate,h_d_totalPPA,h_o_explosiveness,h_o_successrate,h_o_stuffrate,h_o_totalPPA,home_conference,away_conference,neutral_site
0,1446,1442,2.0,13.461538,1.331315,0.387627,0.209011,9.847823,1.249980,0.435223,0.188526,13.564157,0,2,0
1,1449,1665,2.5,12.600000,1.290190,0.413603,0.174861,12.324514,1.239361,0.427519,0.223501,13.328906,10,10,0
2,1542,1480,-10.0,13.750000,1.239019,0.448006,0.202265,15.622856,1.227273,0.392168,0.212778,8.339411,0,0,0
3,1684,1522,-24.0,11.303030,1.441883,0.377862,0.185552,9.415842,1.096152,0.476787,0.130066,17.938538,7,7,0
4,1251,1641,16.0,11.925926,1.238476,0.437122,0.197827,13.253949,1.422501,0.362134,0.238543,5.755227,4,7,0


In [29]:
week4preds = w4xgb.predict(week4)
len(week4preds)

60

In [30]:
predictionsDB2 = predictionsDB[['home_team', 'away_team','spread', 'DTpreds']]
predictionsDB2['XGB_preds'] = week4preds

C:\Users\magamp\AppData\Local\Temp\ipykernel_23544\2357827412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictionsDB2['XGB_preds'] = week4preds


In [31]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(predictionsDB2)

            home_team              away_team  spread  DTpreds  XGB_preds
0       Virginia Tech          West Virginia     2.0        1          1
1       Georgia State       Coastal Carolina     2.5        0          1
2            Syracuse               Virginia   -10.0        0          0
3           Air Force                 Nevada   -24.0        0          0
4                UTEP            Boise State    16.0        1          1
5   Mississippi State          Bowling Green   -30.0        0          0
6          Iowa State                 Baylor    -2.5        1          1
7         Wake Forest                Clemson     7.5        1          1
8            Michigan               Maryland   -17.0        1          1
9              Auburn               Missouri    -7.0        1          1
10             Kansas                   Duke    -7.5        0          0
11                SMU                    TCU     1.5        0          1
12   Eastern Michigan                Buffalo    -6.

In [32]:
predictionsDB2.to_csv('week4_predictions.csv')